#### 🔎 5.6.1 모델 구조

- `torchsummary` : 구조와 모델 변수를 간략히 알려주는 라이브러리

- 입력층 -> 첫번째 은닉층으로 넘어가는 부분의 가중치 : 13x50 = 650개 + 편향 : 50개 = 총 700개의 변수
- 첫번째 -> 두번째 은닉층 사이의 가중치 : 50x30 = 1500개 + 편향 : 30개 = 1530개의 변수
- 마지막 출력층으로 가는 가중치 : 30개 + 편향 : 1개 = 31개의 변수

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary

In [3]:
class Regressor(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.fc1 = nn.Linear(13, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x) :
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        
        return x

- 모델을 출력하면 __init__ 부분에서 정의된 구조 확인 가능

In [4]:
model = Regressor()
print(model)

Regressor(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


--------------------------------------------------------------------------------------------

#### 🔎 5.6.2 모델 변수

##### 💡 `model.parameters() : `를 통해 정의된 순서대로 변수를 얻을 수 있다.

In [9]:
for parameter in model.parameters() :
    print(parameter.size())

torch.Size([50, 13])
torch.Size([50])
torch.Size([30, 50])
torch.Size([30])
torch.Size([1, 30])
torch.Size([1])


##### 💡 변수명을 알고 있다면 직접 접근하여 변수를 불러올 수 있다.
- 예를 들어 model에 fc1의 가중치를 불러오고 싶다면 `model.fc1.weight.size()`
- 편향을 불러오고 싶다면 `model.fc1.bias.size()`

In [8]:
print(model.fc1.weight.size(), model.fc1.bias.size())

torch.Size([50, 13]) torch.Size([50])


##### 💡 변수명을 잘 모를 경우에는 `model.named_parameters()`를 통해 변수명과 변수를 동시에 불러올 수 있다.

In [10]:
for name, param in model.named_parameters() :
    print(name, param.size())

fc1.weight torch.Size([50, 13])
fc1.bias torch.Size([50])
fc2.weight torch.Size([30, 50])
fc2.bias torch.Size([30])
fc3.weight torch.Size([1, 30])
fc3.bias torch.Size([1])


##### 💡 특정 변수에 대한 옵션과 값을 변경할 수 있다.
- torchsummary에서 제공하는 summary에 model을 넣고 임의의 입력 데이터 사이즈를 넣으면 층마다 출력값의 크기와 변수에 대한 정보를 테이블로 만들어준다.

In [23]:
summary(model, (10, 13))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 10, 50]             700
            Linear-2               [-1, 10, 30]           1,530
           Dropout-3               [-1, 10, 30]               0
            Linear-4                [-1, 10, 1]              31
Total params: 2,261
Trainable params: 2,261
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------
